# PROJETO 5 - ANÁLISE DE LESÕES NA PELE COM INTELIGÊNCIA ARTIFICIAL

Neste projeto será feito uma classificação multiclasses de diferentes tipos de lesões na pele humana. Para isso, será usando modelos de deep learning com arquitetura CNN e Densenet. 

Dicionário dos dados: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T

Fonte dos dados: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000

## 1. Instalando e carregando pacotes

In [1]:
# Versão da linguagem Python
from platform import python_version
print('A versão da linguagem Python utilizada neste Jupyter Notebook é: ', python_version())

A versão da linguagem Python utilizada neste Jupyter Notebook é:  3.9.13


In [2]:
# Instala Pytorch
!pip install -q torch==1.13.0

In [3]:
# Instala Torchvision
!pip install -q torchvision==0.14.0

In [4]:
# Instala Lightning
!pip install -q pytorch-lightning==1.8.3

In [5]:
# Importa os pacotes/funções

# Manipulação das imagens
import os # Manipula o Sistema Operacional
import cv2 # Converte imagens em dados
import itertools # Iteração dos dados
from tqdm import tqdm # Barra de progressão
from glob import glob # Manipulação de imagens
from PIL import Image # Manipulação de imagens

# Manipulação e visualização dos dados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Pytorch
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Pacotes para o relatório de hardware
import gc
import types
import pkg_resources
import pytorch_lightning as pl

# Seend para reprodução dos resultados DSA
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Projeto 5 - IA para Análise de Imagens na Pele" --iversions

Author: Projeto 5 - IA para Análise de Imagens na Pele

matplotlib       : 3.5.2
torchvision      : 0.14.0
PIL              : 9.2.0
torch            : 1.13.0
pandas           : 1.4.4
cv2              : 4.7.0
pytorch_lightning: 1.8.3
numpy            : 1.21.5



## 2. Verificando o Hardware

In [7]:
# Relatório completo

# Verificando o dispositivo
processing_device = "cuda" if torch.cuda.is_available() else "cpu"

# Verificando se GPU pode ser usada (isso depende da plataforma CUDA estar instalada)
torch_aval = torch.cuda.is_available()

# Labels para o relatório de verificação
lable_1 = 'Visão Geral do Ambiente'
lable_2 = 'Se NVIDIA-SMI não for encontrado, então CUDA não está disponível'
lable_3 = 'Fim da Checagem'

# Função para verificar o que está importado nesta sessão
def get_imports():

    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):            
            name = val.__module__.split(".")[0]

        poorly_named_packages = {"PIL": "Pillow", "sklearn": "scikit-learn"}

        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

# Imports nesta sessão
imports = list(set(get_imports()))

# Loop para verificar os requerimentos
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))
        
# Pasta com os dados (quando necessário)
pasta_dados = r'dados'

print(f'{lable_1:-^100}')
print()
print(f"Device:", processing_device)
print(f"Pasta de Dados: ", pasta_dados)
print(f"Versões dos Pacotes Requeridos: ", requirements)
print(f"Dispositivo Que Será Usado Para Treinar o Modelo: ", processing_device)
print(f"CUDA Está Disponível? ", torch_aval)
print("Versão do PyTorch: ", torch.__version__)
print("Versão do Lightning: ", pl.__version__)
print()
print(f'{lable_2:-^100}\n')
!nvidia-smi
gc.collect()
print()
print(f"Limpando a Memória da GPU (se disponível): ", torch.cuda.empty_cache())
print("\nModelo da GPU:")
# Modelo da GPU usada
!nvidia-smi --query-gpu=name --format=csv,noheader
print(f'\n{lable_3:-^100}')

--------------------------------------Visão Geral do Ambiente---------------------------------------

Device: cpu
Pasta de Dados:  dados
Versões dos Pacotes Requeridos:  [('Pillow', '9.2.0'), ('tqdm', '4.64.1'), ('matplotlib', '3.5.2'), ('numpy', '1.21.5'), ('torch', '1.13.0'), ('pandas', '1.4.4'), ('torchvision', '0.14.0')]
Dispositivo Que Será Usado Para Treinar o Modelo:  cpu
CUDA Está Disponível?  False
Versão do PyTorch:  1.13.0+cpu
Versão do Lightning:  1.8.3

------------------Se NVIDIA-SMI não for encontrado, então CUDA não está disponível------------------


Limpando a Memória da GPU (se disponível):  None

Modelo da GPU:


'nvidia-smi' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.



------------------------------------------Fim da Checagem-------------------------------------------


'nvidia-smi' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


## 3. Interpretação dos metadados

In [8]:
# Leitura do arquivo de metadados
df_inicial = pd.read_csv(os.path.join('dados', 'HAM10000_metadata.csv'))

# Visualiza o registro
df_inicial.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [9]:
# Caminho para as imagens
caminho_imagens = glob(os.path.join('dados','*','*.jpg'))

In [10]:
# Cria um dicionário com id da imagem e caminho do arquivo no disco
dict_imagens = {os.path.splitext(os.path.basename(x))[0]: x for x in caminho_imagens}

In [11]:
# Imprimindo 5 registros do dicionário
dict(itertools.islice(dict_imagens.items(), 5))

{'ISIC_0024306': 'dados\\HAM10000_images_part_1\\ISIC_0024306.jpg',
 'ISIC_0024307': 'dados\\HAM10000_images_part_1\\ISIC_0024307.jpg',
 'ISIC_0024308': 'dados\\HAM10000_images_part_1\\ISIC_0024308.jpg',
 'ISIC_0024309': 'dados\\HAM10000_images_part_1\\ISIC_0024309.jpg',
 'ISIC_0024310': 'dados\\HAM10000_images_part_1\\ISIC_0024310.jpg'}

In [12]:
# Adiciona o path ao df_inicial
df_inicial['path'] = df_inicial['image_id'].map(dict_imagens.get)

In [13]:
# Visualiza o dataset
df_inicial.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0027419.jpg
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0025030.jpg
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0025661.jpg
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,dados\HAM10000_images_part_2\ISIC_0031633.jpg


In [14]:
# Tipos de lesões que serão analisadas de acordo com o dicionário de dados
tipo_lesao_dict = {'nv': 'Melanocytic nevi',
                   'mel': 'dermatofibroma',
                   'bkl': 'Benign keratosis-like',
                   'bcc': 'Basal cell carcinoma',
                   'akiec': 'Actinic keratoses',
                   'vasc': 'Vascular lesions',
                   'df': 'Dermatofibroma'}

In [15]:
# Extrair o tipo de lesão
df_inicial['tipo_lesao'] = df_inicial['dx'].map(tipo_lesao_dict.get)

In [16]:
# Visualiza o dataset
df_inicial.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,tipo_lesao
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0027419.jpg,Benign keratosis-like
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0025030.jpg,Benign keratosis-like
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0026769.jpg,Benign keratosis-like
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0025661.jpg,Benign keratosis-like
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,dados\HAM10000_images_part_2\ISIC_0031633.jpg,Benign keratosis-like


In [17]:
# Converte a variável categórica em sua representação numérica
df_inicial['tipo_lesao_idx'] = pd.Categorical(df_inicial['tipo_lesao']).codes

In [18]:
# Visualiza o dataset
df_inicial.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,tipo_lesao,tipo_lesao_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0027419.jpg,Benign keratosis-like,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0025030.jpg,Benign keratosis-like,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0026769.jpg,Benign keratosis-like,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,dados\HAM10000_images_part_1\ISIC_0025661.jpg,Benign keratosis-like,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,dados\HAM10000_images_part_2\ISIC_0031633.jpg,Benign keratosis-like,2


## 4. Pré-Processamento dos dados  

### 4.1 Extração da média e desvio padrão

In [19]:
# Função para cálculo de média e desvio
def func_calcula_img_mean_std(image_paths):

    # Define altura e largura que usaremos nas imagens
    # Densenet espera as imagens nesta dimensão
    img_h, img_w = 224, 224
    
    # Listas de controle
    imgs = []
    means, stdevs = [], []

    # Loop de leitura e resize das imagens
    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        imgs.append(img)

    # Stack de imagens
    imgs = np.stack(imgs, axis=3)
    print(imgs.shape)

    # Normalização
    imgs = imgs.astype(np.float32) / 255.

    # Loop de cálculo da média e desvio
    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))

    # BGR --> RGB
    means.reverse()  
    stdevs.reverse()

    print("normMean = {}".format(means))
    print("normStd = {}".format(stdevs))
    
    return means, stdevs

In [20]:
# Retorna a média e o desvio padrão de cada canal RGB
norm_mean, norm_std = func_calcula_img_mean_std(caminho_imagens)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10015/10015 [01:56<00:00, 86.33it/s]


(224, 224, 3, 10015)
normMean = [0.7630331, 0.5456457, 0.5700467]
normStd = [0.1409281, 0.15261227, 0.16997086]


### 4.2 Preparação para o dataset de validação

In [21]:
# Verifica o número de imagens associadas ao lesion_id
df_temp = df_inicial.groupby('lesion_id').count()

# Visualiza o dataset
df_temp.head()

,image_id,dx,dx_type,age,sex,localization,path,tipo_lesao,tipo_lesao_idx
lesion_id,,,,,,,,,
HAM_0000000,2,2,2,2,2,2,2,2,2
HAM_0000001,1,1,1,1,1,1,1,1,1
HAM_0000002,3,3,3,3,3,3,3,3,3
HAM_0000003,1,1,1,1,1,1,1,1,1
HAM_0000004,1,1,1,1,1,1,1,1,1


In [22]:
# Agora filtramos lesion_ids que possuem apenas uma imagem associada
df_temp = df_temp[df_temp['image_id'] == 1]

In [23]:
# Reset do índice
df_temp.reset_index(inplace = True)

In [24]:
# Visualiza o dataset
df_temp.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,tipo_lesao,tipo_lesao_idx
0,HAM_0000001,1,1,1,1,1,1,1,1,1
1,HAM_0000003,1,1,1,1,1,1,1,1,1
2,HAM_0000004,1,1,1,1,1,1,1,1,1
3,HAM_0000007,1,1,1,1,1,1,1,1,1
4,HAM_0000008,1,1,1,1,1,1,1,1,1


In [25]:
# Função para identificar lesion_ids que possuem imagens duplicadas e aqueles que possuem apenas uma imagem
def get_duplicates(x):
    unique_list = list(df_temp['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'

In [26]:
# Cria uma nova coluna que seja uma cópia da coluna lesion_id
df_inicial['duplicates'] = df_inicial['lesion_id']

In [27]:
# Aplica a função a esta nova coluna
df_inicial['duplicates'] = df_inicial['duplicates'].apply(get_duplicates)

In [28]:
# Contagem dos valores duplicados
df_inicial['duplicates'].value_counts()

unduplicated    5514
duplicated      4501
Name: duplicates, dtype: int64

In [29]:
# Filtro das imagens que não têm duplicatas
df_temp = df_inicial[df_inicial['duplicates'] == 'unduplicated']

In [30]:
# Define y
y = df_temp['tipo_lesao_idx']

In [31]:
# Cria dataset de validação sem duplicatas
_, df_val = train_test_split(df_temp, test_size = 0.2, random_state = 101, stratify = y)

In [32]:
# Visualiza o shape do dataset
df_val.shape

(1103, 11)

In [33]:
# Verifica a contagem dos itens
df_val['tipo_lesao_idx'].value_counts()

4    883
2     88
6     46
1     35
0     30
5     13
3      8
Name: tipo_lesao_idx, dtype: int64

### 4.3 Separação das amostras de treino e validação

In [34]:
# Esta função identifica se uma imagem faz parte do conjunto train ou val
def get_val_rows(x):
    val_list = list(df_val['image_id'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

In [35]:
# Identifica treino ou validação
df_inicial['train_or_val'] = df_inicial['image_id']

In [36]:
# Aplica a função a esta nova coluna
df_inicial['train_or_val'] = df_inicial['train_or_val'].apply(get_val_rows)

In [37]:
# Filtra as linhas de treino
df_treino = df_inicial[df_inicial['train_or_val'] == 'train']

In [38]:
# Verifica o número de registros
print('Dataset de treino: ', len(df_treino))
print('Dataset de validação: ', len(df_val))

Dataset de treino:  8912
Dataset de validação:  1103


In [39]:
# Total de itens por classe
# Dataset desbalanceado
df_treino['tipo_lesao_idx'].value_counts()

4    5822
6    1067
2    1011
1     479
0     297
5     129
3     107
Name: tipo_lesao_idx, dtype: int64

In [40]:
# Total de itens por classe
df_val['tipo_lesao'].value_counts()

Melanocytic nevi         883
Benign keratosis-like     88
dermatofibroma            46
Basal cell carcinoma      35
Actinic keratoses         30
Vascular lesions          13
Dermatofibroma             8
Name: tipo_lesao, dtype: int64

### 4.4 Dataset Augmentation

In [41]:
# Taxa de dataset augmentation a ser usada em cada classe
# É determinada de forma manual testando o balancemento no final
data_aug_rate = [15,10,5,50,0,40,5]

In [42]:
# Loop para o dataset augmentation
for i in range(7):
    
    if data_aug_rate[i]:
        
        # Equaliza a proporção de imagens por classe nos dados de treino
        # Geramos novas imagens multiplicando as imagens existentes pela taxa definida na lista de taxas
        df_treino = df_treino.append([df_treino.loc[df_treino['tipo_lesao_idx'] == i,:]] * (data_aug_rate[i] - 1), 
                                     ignore_index = True)

In [43]:
# Visualiza o dataset de treino
df_treino['tipo_lesao'].value_counts()

Melanocytic nevi         5822
Dermatofibroma           5350
dermatofibroma           5335
Vascular lesions         5160
Benign keratosis-like    5055
Basal cell carcinoma     4790
Actinic keratoses        4455
Name: tipo_lesao, dtype: int64

In [44]:
# Reset do índice
df_treino = df_treino.reset_index()

In [45]:
# Shape do dataset de treino
df_treino.shape

(35967, 13)

### 4.5 Preparação das amostras de treino, validação e teste

In [46]:
# Divisão dos dados
df_val, df_teste = train_test_split(df_val, test_size = 0.5)

In [47]:
# Reset do índice
df_val = df_val.reset_index()
df_teste = df_teste.reset_index()

In [48]:
# Shape dos dados

print('Shape dos dados de treino: ', df_treino.shape)
print('Shape dos dados de validação: ', df_val.shape)
print('Shape dos dados de teste: ', df_teste.shape)

Shape dos dados de treino:  (35967, 13)
Shape dos dados de validação:  (551, 12)
Shape dos dados de teste:  (552, 12)


### 4.6 Transformação das Imagens

In [49]:
# Tamanho das imagens de entrada
input_size = 224

In [51]:
# Transformações das imagens de treino
transform_treino = transforms.Compose([transforms.Resize((input_size, input_size)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness = 0.1, contrast = 0.1, hue = 0.1),
                                      transforms.ToTensor(),
                                      transforms.Normalize(norm_mean, norm_std)])

In [53]:
# Transformações das imagens de validação
transform_val = transforms.Compose([transforms.Resize((input_size, input_size)),
                                  transforms.ToTensor(),
                                  transforms.Normalize(norm_mean, norm_std)])

In [54]:
# Define um organizador de dados
class OrganizaDados(Dataset):
    
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['tipo_lesao_idx'][index]))
        
        if self.transform:
            X = self.transform(X)
            
        
        return X, y

In [55]:
# Organiza e transforma os dados de treino
set_treino = OrganizaDados(df_treino, transform = transform_treino)

In [57]:
# Organiza e transforma os dados de validação
set_val = OrganizaDados(df_val, transform = transform_val)

In [58]:
# Organiza e transforma os dados de teste
set_teste = OrganizaDados(df_teste, transform = transform_val)

### 4.7 Criação dos DataLoaders

In [60]:
# Cria o dataloader de treino
loader_treino = DataLoader(set_treino, batch_size = 32, shuffle = True, num_workers = 4)

In [61]:
# Cria o dataloader de validação
loader_val = DataLoader(set_val, batch_size = 32, shuffle = False, num_workers = 4)

In [62]:
# Cria o dataloader de teste
loader_teste = DataLoader(set_teste, batch_size = 32, shuffle = False, num_workers = 4)

## 5. Modelagem - Arquitetura Densenet

A DenseNet, ou Rede Densa Conectada, é uma arquitetura de rede neural convolucional (CNN) proposta por Gao Huang, Zhuang Liu, Laurens van der Maaten e Kilian Q. Weinberger em 2016. 

A ideia principal por trás da DenseNet é promover a conexão direta entre camadas não adjacentes, facilitando o fluxo de informações e gradientes ao longo da rede. Essa arquitetura é especialmente eficiente no que diz respeito à utilização de recursos e melhora a qualidade das representações aprendidas.

link: https://pytorch.org/vision/main/models/generated/torchvision.models.densenet121.html

In [63]:
# Define o número de classes, quantidade de lesões na pele
num_classes = 7

In [64]:
# Carga do modelo pré-treinado com todos os pesos
modelo_densenet = models.densenet121(pretrained = True)

In [65]:
# Define o número de atributos de entrada
num_ftrs = modelo_densenet.classifier.in_features

In [66]:
# Camada linear final para prever a probabilidade das 7 classes
modelo_densenet.fc = nn.Linear(num_ftrs, num_classes)

In [67]:
# Função para definir o uso do modelo pré-treinado
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False